## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-16-05-41-46 +0000,nypost,"Burger King worker, mom of 3 Nykia Hamilton fo...",https://nypost.com/2025/07/16/us-news/burger-k...
1,2025-07-16-05-37-50 +0000,bbc,Driver held for hit-and-run death of world's '...,https://www.bbc.com/news/articles/c2d0yep9d37o
2,2025-07-16-05-22-29 +0000,nypost,Polite black bear stuns New Hampshire homeowne...,https://nypost.com/2025/07/16/us-news/polite-b...
3,2025-07-16-04-56-20 +0000,bbc,Man who murdered British backpacker Peter Falc...,https://www.bbc.com/news/articles/cjrlg4x8e78o
4,2025-07-16-04-46-03 +0000,bbc,S Korea arrests teacher and parent for exam pa...,https://www.bbc.com/news/articles/cq6m17y8g25o


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
132,trump,14
191,up,5
41,dies,4
53,president,4
26,new,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
24,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...,34
43,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...,33
20,2025-07-16-01-00-00 +0000,wsj,The impact of President Trump’s whirlwind six ...,https://www.wsj.com/economy/trump-effect-start...,31
18,2025-07-16-01-27-41 +0000,nypost,Joe Rogan rips DOJ’s handling of Epstein files...,https://nypost.com/2025/07/15/us-news/joe-roga...,30
35,2025-07-15-22-36-03 +0000,latimes,"Trump accuses Schiff of mortgage fraud, which ...",https://www.latimes.com/politics/story/2025-07...,28


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
24,34,2025-07-16-00-00-00 +0000,wsj,"President Trump wants tariffs, the higher the ...",https://www.wsj.com/economy/trade/forget-taco-...
40,23,2025-07-15-20-41-00 +0000,wsj,"Inflation picked up in June, a potential sign ...",https://www.wsj.com/economy/inflation-hit-2-7-...
12,23,2025-07-16-02-33-54 +0000,nypost,Beloved teacher dies in 100-foot fall just 2 m...,https://nypost.com/2025/07/15/us-news/beloved-...
2,20,2025-07-16-05-22-29 +0000,nypost,Polite black bear stuns New Hampshire homeowne...,https://nypost.com/2025/07/16/us-news/polite-b...
1,19,2025-07-16-05-37-50 +0000,bbc,Driver held for hit-and-run death of world's '...,https://www.bbc.com/news/articles/c2d0yep9d37o
43,19,2025-07-15-19-03-00 +0000,wsj,JPMorgan CEO Jamie Dimon sounded Wall Street’s...,https://www.wsj.com/economy/central-banking/di...
15,19,2025-07-16-01-56-15 +0000,nypost,NYC Council rejects land use change for Bally’...,https://nypost.com/2025/07/15/us-news/nyc-coun...
8,17,2025-07-16-03-49-27 +0000,nypost,Vance casts tie-breaking votes to move forward...,https://nypost.com/2025/07/15/us-news/vance-ca...
25,16,2025-07-15-23-50-44 +0000,nypost,Alleged burglar arrested in the shooting death...,https://nypost.com/2025/07/15/us-news/alleged-...
0,14,2025-07-16-05-41-46 +0000,nypost,"Burger King worker, mom of 3 Nykia Hamilton fo...",https://nypost.com/2025/07/16/us-news/burger-k...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
